<a href="https://colab.research.google.com/github/deekshahs1817/Infosys_SpringBoard_All_milestones-1-4-/blob/main/dee_milestone_3_addons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run in a notebook cell (Colab)
!pip install streamlit pyngrok opencv-python-headless pillow pytesseract numpy sentence-transformers tqdm
!apt-get update -qq && apt-get install -y -qq tesseract-ocr exiftool



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.9 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libarchive-zip-perl.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libarchive-zip-perl_1.68-1_all.deb ...
Unpacking libarchive-zip-perl (1.68-1) ...
Selecting previously unselected package libimage-exiftool-perl.
Preparing to unpack .../libimage-exiftool-perl_12.40+dfsg-1_all.deb ...
Unpacking libimage-exiftool-perl (12.40+dfsg-1) ...
Selecting previously unselected package libmime-charset-perl.
Preparing to unpack .../libmime-charset-perl_1.012.2-1_all.deb ...
Unpacking libmime-charset-perl (1.012.2-1) ...
Selecting previously unselected package libsombok3:am

In [ ]:
!pip install pytesseract


In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!apt-get install -y tesseract-ocr-hin


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-hin
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 913 kB of archives.
After this operation, 1,138 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-hin all 1:4.00~git30-7274cfa-1.1 [913 kB]
Fetched 913 kB in 1s (765 kB/s)
Selecting previously unselected package tesseract-ocr-hin.
(Reading database ... 122445 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-hin_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-hin (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-hin (1:4.00~git30-7274cfa-1.1) ...


In [ ]:
!pip install streamlit


In [ ]:
# app.py
# Single-file KYC / Admin / OCR / Fraud Alerts Streamlit app
# Features: login/register, admin, multi-doc KYC (Aadhaar/PAN/Utility/Selfie),
# PAN smart extraction (name under photo, DOB by label), tamper/blur/heatmap,
# browser GPS, selfie EXIF GPS, address similarity (sentence-transformers),
# fingerprint enrollment (image upload, ORB), live alerts, SQLite audits.
# Updated: added "Back to Login" button in register page, improved UI layout,
# and heatmap display in KYC verification results + additional emojis & polish.
%%writefile app.py
import streamlit as st
from PIL import Image, ExifTags
import io, base64, time, json, sqlite3, re, os, hashlib, hmac, sys, traceback
import numpy as np
import cv2
import streamlit.components.v1 as components
from difflib import SequenceMatcher

# Try to import heavy libs and handle missing ones gracefully
MISSING = []
try:
    import pytesseract
except Exception:
    MISSING.append("pytesseract")
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
except Exception:
    # If missing, vector similarity will be disabled with fallback string ratio
    SentenceTransformer = None
    cosine_similarity = None
    MISSING.append("sentence-transformers / scikit-learn")

# ---------- CONFIG ----------
DB_PATH = "/content/kyc_local.db"
SALT = b"kyc_app_salt_v1"   # change in production
EMBED_MODEL = None

# ---------- UTIL: DB ----------
def init_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password_hash TEXT,
        role TEXT,
        created_at TEXT
    )""")
    c.execute("""
    CREATE TABLE IF NOT EXISTS audits (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        document_id TEXT,
        payload TEXT,
        timestamp TEXT
    )""")
    c.execute("""
    CREATE TABLE IF NOT EXISTS fingerprints (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        account_id TEXT,
        descriptor BLOB,
        created_at TEXT
    )""")
    conn.commit()
    conn.close()

init_db()

# ---------- AUTH HELPERS ----------
def hash_password(password: str) -> str:
    dk = hashlib.pbkdf2_hmac("sha256", password.encode(), SALT, 200000)
    return base64.b64encode(dk).decode()

def verify_password(password: str, stored_hash: str) -> bool:
    dk = hashlib.pbkdf2_hmac("sha256", password.encode(), SALT, 200000)
    return hmac.compare_digest(base64.b64encode(dk).decode(), stored_hash)

def create_user(username, password, role="user"):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    ph = hash_password(password)
    try:
        c.execute("INSERT INTO users (username,password_hash,role,created_at) VALUES (?,?,?,?)",
                  (username, ph, role, time.strftime("%Y-%m-%d %H:%M:%S")))
        conn.commit(); ok = True
    except Exception:
        ok = False
    conn.close(); return ok

def get_user_row(username):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("SELECT id,username,role,password_hash,created_at FROM users WHERE username=?", (username,))
    row = c.fetchone(); conn.close(); return row

def list_users():
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("SELECT id,username,role,created_at FROM users ORDER BY id")
    rows = c.fetchall(); conn.close(); return rows

def set_role(username, role):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("UPDATE users SET role=? WHERE username=?", (role, username)); conn.commit(); conn.close()

def delete_user(username):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("DELETE FROM users WHERE username=?", (username,)); conn.commit(); conn.close()

# ensure admin exists
def ensure_admin():
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("SELECT COUNT(*) FROM users WHERE role='admin'"); cnt = c.fetchone()[0]
    if cnt == 0:
        create_user("admin", "admin123", role="admin")
    conn.close()

ensure_admin()

# ---------- AUDITS ----------
def save_audit(doc_id, payload):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("INSERT INTO audits (document_id,payload,timestamp) VALUES (?,?,?)",
              (doc_id, json.dumps(payload), time.strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit(); conn.close()

def read_audits(limit=50):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("SELECT * FROM audits ORDER BY id DESC LIMIT ?", (limit,))
    rows = c.fetchall(); conn.close(); return rows

# ---------- FINGERPRINT (image-upload ORB) ----------
def extract_fp_descriptors(cv_img):
    orb = cv2.ORB_create(500)
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    kps, des = orb.detectAndCompute(gray, None)
    return des

def store_fingerprint(account_id, des):
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("INSERT INTO fingerprints (account_id, descriptor, created_at) VALUES (?,?,?)",
              (account_id, des.tobytes(), time.strftime("%Y-%m-%d")))
    conn.commit(); conn.close()

def load_fingerprints():
    conn = sqlite3.connect(DB_PATH); c = conn.cursor()
    c.execute("SELECT account_id, descriptor FROM fingerprints")
    rows = c.fetchall(); conn.close()
    result = []
    for acc, blob in rows:
        try:
            arr = np.frombuffer(blob, dtype=np.uint8).reshape((-1,32))
            result.append((acc, arr))
        except Exception:
            continue
    return result

def match_fp(des1, des2):
    if des1 is None or des2 is None: return 0.0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    good = [m for m in matches if m.distance < 50]
    score = len(good) / max(1, min(len(des1), len(des2)))
    return score

# ---------- OCR Patterns ----------
PHONE_RE = re.compile(r"[6-9]\d{9}")
AADHAAR_RE = re.compile(r"\d{4}\s?\d{4}\s?\d{4}")
PAN_RE = re.compile(r"[A-Z]{5}\d{4}[A-Z]")
DOB_RE = re.compile(r"\d{2}[/-]\d{2}[/-]\d{4}")
PIN_RE = re.compile(r"\b\d{6}\b")

# ---------- PAN extractor & OCR ----------
def extract_pan_details(lines, text):
    pan_m = PAN_RE.search(text)
    pan_num = pan_m.group() if pan_m else "Not detected"
    caps = [l for l in lines if l.isupper() and len(l.replace(" ", "")) > 3]
    name = caps[0] if caps else next((l for l in lines if len(l) > 3 and not any(ch.isdigit() for ch in l)), "Not detected")
    father_name = "Not detected"
    if name in lines:
        idx = lines.index(name)
        if idx + 1 < len(lines): father_name = lines[idx+1]
    dob = "Not detected"
    for i,l in enumerate(lines):
        if "date of birth" in l.lower() or "dob" in l.lower():
            dm = DOB_RE.search(l)
            if dm: dob = dm.group(); break
            elif i+1 < len(lines):
                dm2 = DOB_RE.search(lines[i+1])
                if dm2: dob = dm2.group(); break
    return {"pan_number": pan_num, "name": name, "father_name": father_name, "dob": dob}

def run_ocr_single(pil_img, doc_type=None):
    # returns dict of extracted fields for that single PIL image
    try:
        text = pytesseract.image_to_string(pil_img)
    except Exception as e:
        return {"error": f"pytesseract error: {e}"}
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    name_guess = next((l for l in lines if len(l) > 3 and not any(ch.isdigit() for ch in l)), "Not detected")
    phone_m = PHONE_RE.search(text); phone = phone_m.group() if phone_m else "Not detected"
    aadhaar_m = AADHAAR_RE.search(text); aadhaar = aadhaar_m.group() if aadhaar_m else "Not detected"
    pan_m = PAN_RE.search(text); pan_val = pan_m.group() if pan_m else "Not detected"
    dob_m = DOB_RE.search(text); dob_val = dob_m.group() if dob_m else "Not detected"
    if doc_type == "pan" or pan_m:
        return_pan = extract_pan_details(lines, text)
        return {
            "raw_text": text,
            "name": return_pan["name"],
            "father_name": return_pan["father_name"],
            "phone": phone,
            "aadhaar": aadhaar,
            "pan": return_pan["pan_number"],
            "dob": return_pan["dob"],
            "address": "Not detected"
        }
    # Aadhaar/utility fallback address extraction
    address = "Not detected"
    addr_prefixes = ["s/o","d/o","c/o","w/o","#"]
    start_idx = None
    for i,l in enumerate(lines):
        if any(l.lower().startswith(p) for p in addr_prefixes):
            start_idx = i; break
    if start_idx is not None:
        collected = []
        for j in range(start_idx, len(lines)):
            if len(lines[j]) < 2: break
            collected.append(lines[j])
        address = " ".join(collected)
    else:
        pinm = PIN_RE.search(text)
        if pinm:
            p = pinm.group()
            for i,l in enumerate(lines):
                if p in l:
                    address = " ".join(lines[max(0,i-3):i+1]); break
    return {
        "raw_text": text,
        "name": name_guess,
        "phone": phone,
        "aadhaar": aadhaar,
        "pan": pan_val,
        "dob": dob_val,
        "address": address
    }

def run_ocr_multiple(uploaded_files, doc_type=None):
    # uploaded_files: list of UploadedFile or raw bytes
    outputs = []
    for f in uploaded_files:
        try:
            if isinstance(f, bytes):
                pil = Image.open(io.BytesIO(f)).convert("RGB")
            else:
                pil = Image.open(f).convert("RGB")
        except Exception:
            continue
        out = run_ocr_single(pil, doc_type=doc_type)
        outputs.append({"ocr": out, "heatmap": None, "tamper": None, "blur": None})
    return outputs

# ---------- tamper / blur / heat ----------
def tamper_and_heat(cv_img):
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    tamper_score = float(np.mean(edges)/255)
    blur_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    blur_score = max(0, min(1, 1 - blur_var/500))
    heat = cv2.applyColorMap(edges, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(cv_img, 0.6, heat, 0.4, 0)
    _, buf = cv2.imencode(".png", overlay)
    heat_b64 = base64.b64encode(buf).decode()
    return tamper_score, blur_score, heat_b64

# ---------- selfie EXIF GPS ----------
def extract_exif_gps(pil):
    try:
        exif = pil._getexif()
        if not exif: return None
        gps = exif.get(34853)
        if not gps: return None
        def frac(v): return float(v[0]) / float(v[1])
        lat = frac(gps[2][0]) + frac(gps[2][1])/60 + frac(gps[2][2])/3600
        if gps[1] != "N": lat = -lat
        lon = frac(gps[4][0]) + frac(gps[4][1])/60 + frac(gps[4][2])/3600
        if gps[3] != "E": lon = -lon
        return {"latitude": lat, "longitude": lon}
    except Exception:
        return None

# ---------- address similarity ----------
def get_embedder():
    global EMBED_MODEL
    if EMBED_MODEL is None:
        if SentenceTransformer is None:
            EMBED_MODEL = None
        else:
            EMBED_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
    return EMBED_MODEL

def address_similarity(a,b):
    if not a or not b or a == "Not detected" or b == "Not detected":
        return 0.0
    model = get_embedder()
    if model is None:
        # fallback to SequenceMatcher ratio if embedder missing
        return SequenceMatcher(None, a.lower(), b.lower()).ratio()
    v1 = model.encode([a])[0].reshape(1,-1)
    v2 = model.encode([b])[0].reshape(1,-1)
    return float(cosine_similarity(v1, v2)[0][0])

def match_ratio(a,b):
    if not a or not b: return 0.0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# ---------- browser GPS via JS ----------
def request_browser_gps():
    gps_js = """
    <script>
    navigator.geolocation.getCurrentPosition(
      (pos) => { window.parent.postMessage({type:'gps', value: pos.coords}, "*"); },
      (err) => { window.parent.postMessage({type:'gps_error', value: err.message}, "*"); },
      {enableHighAccuracy:true}
    );
    </script>
    """
    components.html(gps_js, height=0)

def process_js_events():
    evt = st.session_state.get("_JS_EVENT")
    if evt:
        if evt.get("type") == "gps":
            st.session_state["gps"] = {"latitude": evt["value"]["latitude"], "longitude": evt["value"]["longitude"], "accuracy": evt["value"]["accuracy"]}
        elif evt.get("type") == "gps_error":
            st.session_state["gps_error"] = evt.get("value")
st.experimental_js = process_js_events

# ---------- auto-refresh ----------
def auto_refresh(seconds=5):
    script = f"""<script>setTimeout(()=>window.parent.postMessage({{'type':'rerun'}}, "*"), {seconds*1000});</script>"""
    components.html(script, height=0)

# ---------- Streamlit UI & routing ----------
st.set_page_config(layout="wide", page_title="KYC Fraud System 🛡️", initial_sidebar_state="expanded")
if "logged_in" not in st.session_state: st.session_state["logged_in"] = False
if "user" not in st.session_state: st.session_state["user"] = None
if "show_register" not in st.session_state: st.session_state["show_register"] = False
if "_admin_open" not in st.session_state: st.session_state["_admin_open"] = False

# ---------- Auth UI ----------
def login_page_ui():
    st.title("🗝️ KYC System — Login")
    if MISSING:
        st.warning("Missing packages: " + ", ".join(MISSING) + ". Install them before heavy features.")
    col1, col2 = st.columns([1,2])
    with col1:
        st.image("https://img.icons8.com/ios-filled/100/000000/lock--v1.png", width=80)
    with col2:
        username = st.text_input("Username", key="login_user")
        password = st.text_input("Password", type="password", key="login_pass")
        c1, c2 = st.columns([1,1])
        with c1:
            if st.button("🔐 Login"):
                row = get_user_row(username)
                if not row:
                    st.error("Invalid username or password ❌")
                    return
                uid, uname, role, pwhash, created = row
                if verify_password(password, pwhash):
                    st.session_state["logged_in"] = True
                    st.session_state["user"] = {"username": uname, "role": role}
                    st.success(f"Logged in as {uname} ({role}) ✅")
                    st.rerun()
                else:
                    st.error("Invalid username or password ❌")
        with c2:
            if st.button("📝 Register"):
                st.session_state["show_register"] = True

def register_page_ui():
    st.title("✍️ Register — Create Account")
    st.info("Choose a username & password. Admin accounts have extra privileges. ⚠️ Use responsibly.")
    username = st.text_input("Choose username", key="reg_user")
    password = st.text_input("Choose password", type="password", key="reg_pass")
    password2 = st.text_input("Repeat password", type="password", key="reg_pass2")
    role = st.selectbox("Role", ["user","admin"], index=0)
    c1, c2 = st.columns([1,1])
    with c1:
        if st.button("✅ Create account"):
            if not username or not password:
                st.error("Provide username & password"); return
            if password != password2:
                st.error("Passwords don't match"); return
            ok = create_user(username, password, role)
            if ok:
                st.success("Account created. Please login. 🎉")
                st.session_state["show_register"] = False
                # small friendly hint
                st.balloons()
            else:
                st.error("Username exists or error creating account ❗")
    with c2:
        # ➜ NEW BUTTON: Go back to Login 🏠
        if st.button("⬅️ Back to Login"):
            st.session_state["show_register"] = False
            st.rerun()

def logout():
    st.session_state["logged_in"] = False
    st.session_state["user"] = None
    st.rerun()

# ---------- Admin ----------
def admin_panel_ui():
    st.header("🧑‍💼 Admin Panel — Users")
    rows = list_users()
    for r in rows:
        st.write(f"{r[0]} | **{r[1]}** | {r[2]} | {r[3]}")
        c1,c2,c3 = st.columns([1,1,1])
        with c1:
            if st.button(f"Make Admin {r[1]}", key=f"madmin_{r[1]}"):
                set_role(r[1], "admin"); st.success(f"{r[1]} is now admin 🎯"); st.rerun()
        with c2:
            if st.button(f"Make User {r[1]}", key=f"muser_{r[1]}"):
                set_role(r[1], "user"); st.success(f"{r[1]} downgraded to user"); st.rerun()
        with c3:
            if st.button(f"Delete {r[1]}", key=f"del_{r[1]}"):
                delete_user(r[1]); st.warning(f"Deleted user {r[1]}"); st.rerun()
    st.markdown("---")
    st.header("📜 Recent Audits")
    audits = read_audits(200)
    for a in audits[:100]:
        st.write(f"{a[0]} | {a[3]}")
        with st.expander("details"):
            try:
                st.json(json.loads(a[2]))
            except Exception:
                st.text(a[2])

# ---------- KYC Dashboard ----------
def kyc_dashboard_ui():
    st.title("🛡️ Advanced KYC Verification + Fraud Analytics")
    st.caption("Multi-document verification • tamper/blur/heatmap • selfie geo-check • fingerprint matching")
    st.write(f"User: **{st.session_state['user']['username']}**  |  Role: **{st.session_state['user']['role']}**")
    if st.button("Logout 🔓"): logout()

    col_left, col_right = st.columns([3,1])

    with col_left:
        st.header("📄 Multi-Document Verification (multiple images allowed)")
        selected = st.multiselect("Select documents to verify:", ["Aadhaar","PAN","Utility Bill","Selfie"], default=["Aadhaar","Selfie"], help="Choose which documents to upload for verification. You can upload multiple images per document.")

        entered_name = entered_phone = entered_address = ""
        if "Aadhaar" in selected:
            st.subheader("Enter Claimed Aadhaar Details 🧾")
            entered_name = st.text_input("Full Name", key="ename")
            entered_phone = st.text_input("Phone Number", key="ephone")
            entered_address = st.text_area("Full Address", key="eaddr")

        st.subheader("Upload Documents 📁 (you can add multiple images per doc)")
        files = {}
        if "Aadhaar" in selected:
            files["aadhaar"] = st.file_uploader("Upload Aadhaar Images", type=["jpg","png","jpeg"], accept_multiple_files=True, key="faad")
        if "PAN" in selected:
            files["pan"] = st.file_uploader("Upload PAN Images", type=["jpg","png","jpeg"], accept_multiple_files=True, key="fpan")
        if "Utility Bill" in selected:
            files["utility"] = st.file_uploader("Upload Utility Bill Images", type=["jpg","png","jpeg"], accept_multiple_files=True, key="futil")
        if "Selfie" in selected:
            files["selfie"] = st.file_uploader("Upload Selfie Images", type=["jpg","png","jpeg"], accept_multiple_files=True, key="fself")

        st.subheader("📍 Browser GPS")
        request_browser_gps()
        gps = st.session_state.get("gps")
        if gps:
            st.success("Browser GPS captured ✅")
            st.json(gps)
        elif st.session_state.get("gps_error"):
            st.error("GPS Error: " + str(st.session_state.get("gps_error")))

        if st.button("Verify KYC Now 🔎"):
            results = {}
            # process each document type
            for doc, uploaded in files.items():
                if not uploaded:
                    results[doc] = {"error": "not uploaded"}
                    continue
                doc_entries = []
                for f in uploaded:
                    try:
                        b = f.read()
                        pil = Image.open(io.BytesIO(b)).convert("RGB")
                    except Exception:
                        continue
                    cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
                    if doc == "pan":
                        ocr = run_ocr_single(pil, doc_type="pan")
                    elif doc == "selfie":
                        ocr = {"exif": extract_exif_gps(pil)}
                    else:
                        ocr = run_ocr_single(pil, doc_type=None)
                    tamper_score = blur_score = heat = None
                    if doc != "selfie":
                        tamper_score, blur_score, heat = tamper_and_heat(cv_img)
                    doc_entries.append({"ocr": ocr, "tamper": tamper_score, "blur": blur_score, "heatmap": heat})
                results[doc] = doc_entries

            # pick primary doc to compare claimed data -> prefer Aadhaar then PAN then Utility
            primary = None
            for p in ("aadhaar","pan","utility"):
                if p in results and isinstance(results[p], list) and len(results[p])>0 and isinstance(results[p][0]["ocr"], dict):
                    primary = results[p][0]["ocr"]; break

            name_score = phone_score = addr_score = 0.0
            if primary:
                name_score = match_ratio(entered_name, primary.get("name",""))
                phone_score = 1.0 if entered_phone and entered_phone == primary.get("phone","") else 0.0
                addr_score = address_similarity(entered_address or "", primary.get("address",""))

            # risk: aggregate worst-case tamper/blur
            risk = 0.0; reasons = []
            for d in ("aadhaar","pan","utility"):
                if d in results and isinstance(results[d], list):
                    for item in results[d]:
                        ts = item.get("tamper")
                        bs = item.get("blur")
                        if ts is not None:
                            if ts > 0.6: risk += 0.4; reasons.append(f"{d} tampering high")
                            elif ts > 0.3: risk += 0.2; reasons.append(f"{d} tampering med")
                        if bs is not None:
                            if bs > 0.6: risk += 0.25; reasons.append(f"{d} blurred")

            risk += (1 - name_score) * 0.3
            risk += (1 - addr_score) * 0.3
            if phone_score == 0:
                risk += 0.3; reasons.append("Phone mismatch")

            # selfie exif vs browser GPS
            if "selfie" in results and gps:
                for item in results["selfie"]:
                    ex = item["ocr"].get("exif") if isinstance(item["ocr"], dict) else None
                    if ex:
                        import math
                        def hav(lat1,lon1,lat2,lon2):
                            R=6371
                            dlat=math.radians(lat2-lat1); dlon=math.radians(lon2-lon1)
                            a=math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
                            return 2*R*math.atan2(math.sqrt(a), math.sqrt(1-a))
                        dist = hav(ex["latitude"], ex["longitude"], gps["latitude"], gps["longitude"])
                        if dist > 5:
                            risk += 0.4; reasons.append(f"selfie geo mismatch {dist:.1f} km")

            risk = min(1.0, max(0.0, risk))
            if risk > 0.6: level = "High"
            elif risk > 0.3: level = "Medium"
            else: level = "Low"

            out = {"documents": results, "scores": {"name": name_score, "phone": phone_score, "address": addr_score}, "risk": risk, "risk_level": level, "reasons": reasons, "gps": gps}
            st.subheader("KYC Result")
            # enhanced metric display with emoji
            risk_emoji = "🟩"
            if level == "High": risk_emoji = "🟥"
            elif level == "Medium": risk_emoji = "🟧"
            st.metric("Fraud Risk", f"{risk_emoji} {level}", f"{risk:.2f}")
            st.json(out)
            save_audit(f"kyc_{time.time()}", out)
            st.success("KYC verification completed ✅", icon="✅")

            # ➜ NEW: Display heatmap images for each document 🔥🖼️ (improved UI)
            st.markdown("----")
            st.subheader("📸 Heatmap Evidence (Tamper Detection) 🔥")
            any_heat = False
            for doc, entries in results.items():
                if isinstance(entries, list):
                    for i, item in enumerate(entries):
                        heat = item.get("heatmap")
                        if heat:
                            any_heat = True
                            st.markdown(f"**{doc.upper()} — Image {i+1}**")
                            try:
                                img_bytes = base64.b64decode(heat)
                                st.image(img_bytes, caption=f"{doc} heatmap 🔍", use_column_width=True)
                            except Exception:
                                st.text("Unable to render heatmap image.")
            if not any_heat:
                st.info("No heatmap images available for uploaded documents.")

    with col_right:
        st.header("🔔 Live Fraud Alerts")
        auto_refresh(5)
        alerts = read_audits(limit=50)
        for r in alerts[:30]:
            try:
                data = json.loads(r[2])
            except Exception:
                data = {"raw": r[2]}
            lvl = data.get("risk_level") or data.get("level") or ("High" if data.get("risk",0)>0.6 else "Medium" if data.get("risk",0)>0.3 else "Low")
            emoji = "🟩"
            if lvl == "High": emoji = "🟥"
            elif lvl == "Medium": emoji = "🟧"
            st.markdown(f"**{emoji} {lvl}** — {r[3]}")
            with st.expander("Details"):
                st.json(data)

    # admin quick button
    if st.session_state["user"]["role"] == "admin":
        if st.button("Open Admin Panel ⚙️"):
            st.session_state["_admin_open"] = True

# ---------- Admin page ----------
def admin_page_ui():
    st.title("🧩 Admin Dashboard")
    if st.button("⬅️ Back to KYC Dashboard"):
        st.session_state["_admin_open"] = False
    admin_panel_ui()

# ---------- Fingerprint page (separate tab support inside admin or user) ----------
def fingerprint_page_ui():
    st.header("🖐️ Fingerprint Enrollment & Matching")
    st.caption("Upload clear fingerprint images (single-finger scans or rolled prints).")
    fpfiles = st.file_uploader("Upload fingerprint images", type=["jpg","png","jpeg"], accept_multiple_files=True)
    acc = st.text_input("Account ID (for enroll/match)")
    enroll = st.checkbox("Enroll uploaded fingerprints to account")
    if st.button("Process Fingerprints"):
        if not fpfiles:
            st.error("Upload one or more fingerprint images")
            return
        stored = load_fingerprints()
        for f in fpfiles:
            b = f.read()
            pil = Image.open(io.BytesIO(b)).convert("RGB")
            cvimg = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
            des = extract_fp_descriptors(cvimg)
            best = 0.0; best_acc = None
            for sacc, sdes in stored:
                s = match_fp(des, sdes)
                if s > best: best = s; best_acc = sacc
            if enroll:
                if not acc:
                    st.error("Provide account id to enroll"); return
                store_fingerprint(acc, des); st.success("Enrolled fingerprint for account "+acc)
            st.metric("Best match score", f"{best:.2f}")
            if best > 0.35:
                st.success(f"Matched with account {best_acc}")
            else:
                st.info("No strong match")

# ---------- main routing ----------
def main():
    # display missing package warnings
    if MISSING:
        st.sidebar.warning("Missing packages: " + ", ".join(MISSING) + ". Install them for full features (OCR/embeddings).")

    if not st.session_state["logged_in"]:
        if st.session_state["show_register"]:
            register_page_ui()
        else:
            login_page_ui()
    else:
        # either admin page open or kyc dashboard
        if st.session_state.get("_admin_open"):
            admin_page_ui()
        else:
            # top navigation tabs
            options = ["KYC Dashboard","Fingerprint","Alerts"]
            if st.session_state["user"]["role"]=="admin":
                options.append("Admin")
            tab = st.sidebar.selectbox("Go to", options)
            if tab == "KYC Dashboard":
                kyc_dashboard_ui()
            elif tab == "Fingerprint":
                fingerprint_page_ui()
            elif tab == "Alerts":
                # show audits
                st.header("📚 All Audits")
                rows = read_audits(200)
                for r in rows:
                    st.write("---")
                    st.write(f"ID: {r[0]}  Time: {r[3]}")
                    with st.expander("Payload"):
                        try:
                            st.json(json.loads(r[2]))
                        except:
                            st.text(r[2])
            elif tab == "Admin":
                admin_page_ui()

if __name__ == "__main__":
    try:
        main()
    except Exception:
        st.error("Fatal error: see logs 🛑")
        tb = traceback.format_exc()
        st.text(tb)


Writing app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.headless true





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.10.194.237:8501

  Stopping...
  Stopping...


In [ ]:
!pip install pyngrok


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
from pyngrok import ngrok
ngrok.kill()


In [ ]:
!pkill -f ngrok


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("36Qu9bgc0UlJm9PIWRmYG63fVVd_4BrsHqQFcornADrZcgR1t")

!nohup streamlit run app.py --server.port 8501 --server.headless true &

public_url = ngrok.connect(8501)
public_url


nohup: appending output to 'nohup.out'


<NgrokTunnel: "https://cf2efff2bc75.ngrok-free.app" -> "http://localhost:8501">